In [ ]:
import os
op = os.path
opd = op.dirname
opj = op.join
orig_dir = os.getcwd()

In [ ]:
os.chdir( opj( opd( opd( orig_dir ) ) ) )

In [ ]:
%matplotlib
##
from glob import glob
from pathlib import Path
from time import sleep, time
import yaml
import re
import numpy as np
from matplotlib import pyplot as plt
import cv2
##
from modules.devices.Olympuslx81 import OLYMP
from modules.devices.Prior import PRIOR
from modules.devices.Evolve512 import EVOLVE
###
from tensorflow.keras import models
import tensorflow.keras as tfk

In [ ]:
ol = OLYMP()
pr = PRIOR()
ev = EVOLVE()

In [ ]:
mod_fold = 'Z:/Lionel_Chiron/Projects/AFML/fast_AFML/models_sets/models_dec4th2022/*'
lm = glob(mod_fold)
# print(lm)

## Load the models

In [ ]:
lm_sorted = sorted(lm, key= lambda elem : int(re.findall('lev(-?\\d+)', elem)[0]))
print(f'lm_sorted { lm_sorted }')
list_mod = []
t0 = time()
print('loading the models for fast AFML.. ')
for m in lm_sorted:
    list_mod += [models.load_model(m)]
t1 = time()
# print(f'time elapsed for loading the models is { round((t1-t0)/60,1) } min')

In [ ]:
def take_pic_posz(i, val, bpp=8, move_type='d', show=False):
    '''
    take the picture at pos posz
    '''
    ol.go_zpos( val, move_type=move_type )            # 797300 
    answ = ol.receive(13)
    print(f'answ 0 {answ}')
    #addr_img = f'uu{i}.tiff'
    addr_img = f'bf_for_optimz{i}.tiff'
    ev.take_pic(addr=addr_img, bpp=bpp, exp_time=100)
#     ev.handle_contrast(True, bpp)
    ev.adapt(bpp)
    if show: plt.imshow(ev.frame, cmap='gray')
    return ev.frame

In [ ]:
def make_pred(i, mod=None, thresh=127, show=False):
    '''
    make the prediction and return surface 
    '''
    img = cv2.imread(f'bf_for_optimz{i}.tiff')
    arr = np.array([ img ], dtype=np.float32)/255                        # format array for prediction
    pred = mod.predict(arr)[0]*255
    ##
    ll = len( np.where( pred > thresh )[0] )
    #print(f'll { ll }')
    addr_pred = f'pred_uu{i}.png'
    cv2.imwrite(addr_pred, pred)
    if show:
        plt.figure()
        plt.imshow(pred, cmap='gray')
    return ll

In [ ]:
def find_z_optim(z, fill_dic_surf=False):
    '''
    find the optimal focus
    z: z position eg: 785400
    '''
    t0 = time()
    step = 20
    pic0 = take_pic_posz(0, z , bpp=8, move_type='d')
    pic1 = take_pic_posz(1, z + step, bpp=8, move_type='d')
    ldiff = []
    print('--------')
    optim = 0
    max_surf = 0
    # Applying the models.. 
    for num, m in enumerate(list_mod):
        surf0 = make_pred(0, mod=m, thresh=127, show=False)
        surf1 = make_pred(1, mod=m, thresh=127, show=False)
        diff = surf1-surf0
        if max_surf < surf0:
            max_surf = surf0
        ldiff += [[surf0, diff]]
        print(f'for mod {num}, diff is {diff}')
        if fill_dic_surf:
            try:
                dic_surf[num] += [surf0]
            except:
                dic_surf[num] = [surf0]
    print(f'max_surf = { max_surf }')
    for mind,(surf,slope) in enumerate(ldiff):
        if surf > max_surf/5 and mind>0:
            if np.sign(slope) == -np.sign(old_slope):
                if np.abs(slope)< np.abs(old_slope):
                    optim = mind
                else:
                    optim = mind-1
                break
        old_slope = slope
            
    
    print(f'***** optim is {optim} *****')
    t1 = time()
    print(f'time elapsed is { round((t1-t0),1) } s')
    return optim

In [ ]:
def plot_curves(lcurves):
    '''
    '''
    lposz, lposz_corr, loptim, lsurf0, ldiff2, z, step = lcurves
    plt.plot(lposz, label='input')
    plt.plot(lposz_corr, label='correction')
    plt.legend()
    print('finished..')
    date = 'dec6th2022'
    plt.savefig(f'meth_13mod_stability_zmid{z}_step{step}_{date}.png')
    plt.figure()
    plt.ylabel('optim')
    plt.plot(loptim)
    plt.savefig(f'list_optim_zmid{z}_step{step}_{date}.png')
    plt.figure()
    plt.ylabel('surf model 6')
    plt.plot(lsurf0)
    plt.savefig(f'list_mod0_zmid{z}_step{step}_{date}.png')
    plt.figure()
    plt.ylabel('diff model 6')
    plt.plot(ldiff2)
    plt.savefig(f'list_ldiff2_zmid{z}_step{step}_{date}.png')

In [ ]:
def test_focus_conservation(z,nb_steps=10):
    '''
    '''
    lposz = []
    lposz_corr = []
    loptim = []
    lsurf0 = []
    ldiff2 = []
    step = 5
    delta_mod = 10   # dist from one model to the other.. 
    for j in range(nb_steps):
        print(f'######################## j is {j}')
        currz = z + (j-nb_steps//2)*step
        for i in range(3):
            ol.go_zpos( currz, move_type='d' )
        optim = find_z_optim(currz)
        loptim += [optim]
        ref = (len(list_mod)-1)//2
        posz_corr = currz-(optim-ref)*delta_mod #+ 2*(10-optim)
        print(f'##################### posz_corr = {posz_corr}')
        for i in range(3):
            ol.go_zpos( posz_corr, move_type='d' )
        take_pic_posz(3+j, posz_corr, bpp=8, move_type='d')
        lposz += [currz]
        lposz_corr += [posz_corr]
#         num_img = (3+j)+100
#         addr_img = f'bf_for_optimz{num_img}.tiff'
#         ev.take_pic(addr=addr_img, bpp=8, exp_time=100)
        surf0 = make_pred(3+j, mod=list_mod[6], thresh=127, show=False)
        lsurf0 +=[surf0]
        take_pic_posz(3+j+100, posz_corr + step, bpp=8, move_type='d')
        surf1 = make_pred(3+j+100, mod=list_mod[6], thresh=127, show=False)
        diff = surf1-surf0
        ldiff2 += [diff]
    lcurves = [lposz, lposz_corr, loptim, lsurf0, ldiff2, z, step]
    plot_curves(lcurves)

In [ ]:
test_focus_conservation(830230,nb_steps=30)

## make the profiles with zstack

In [ ]:
dic_surf = {}
loptim = []
nbstep = 40
for i in range(0,nbstep):  # 40
    print(f'############################## i = {i}')
    optim = find_z_optim(830361+(i-nbstep//2)*10,fill_dic_surf=True) # 833600
    loptim += [optim]
print('finished')

In [ ]:
dir(ol)

In [ ]:
ev.retrieve_autocontrast()

## plot the profiles and predictions.. 

In [ ]:
allmax = 0
for k,v in dic_surf.items():
    print(k)
    maxsurf = max(dic_surf[k])
    if allmax<maxsurf:
        allmax=maxsurf
fact_norm = (len(list_mod)-1)/allmax
for mind, (k,v) in enumerate(dic_surf.items()):
    print(k)
    plt.plot(np.array(dic_surf[k])*fact_norm, alpha=0.3, label=f'lev{mind}')
# for pt in loptim:
#     plt.plot(pt,'rx')
plt.legend()
loptim.insert(0,0)
plt.plot(loptim,'rx')
    
plt.show()

##  Controls

In [ ]:
print(ol.ask_zpos())

In [ ]:
for k,v in dic_surf.items():
    print(k)
    if k!='max':
        plt.plot(np.array(dic_surf[k]))

In [ ]:
find_z_optim(834570) # 833812

In [ ]:
curr_pos = 830250
for i in range(3):
    ol.go_zpos( curr_pos, move_type='d' )

In [ ]:
find_z_optim(curr_pos)

In [ ]:
for i in range(3):
    take_pic_posz(301, curr_pos-10 , bpp=8, move_type='d')

In [ ]:
def aa(ll):
    a, b, c = ll
    print(a,b,c)
aa([1,6,7])